# Importing Libraries

In [1]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data Loading and Initial Inspection

In [2]:
pd_raw_count = pd.read_csv('../data/uk_data/dft_traffic_counts_raw_counts.csv')

/var/folders/_z/nz5spzqj2rl11n5lc_k1cwwc0000gn/T/ipykernel_8447/109602175.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_raw_count = pd.read_csv('../data/uk_data/dft_traffic_counts_raw_counts.csv')


In [3]:
pd_raw_count.shape

(4960860, 35)

In [4]:
pd_raw_count.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,region_ons_code,local_authority_id,local_authority_name,...,buses_and_coaches,LGVs,HGVs_2_rigid_axle,HGVs_3_rigid_axle,HGVs_4_or_more_rigid_axle,HGVs_3_or_4_articulated_axle,HGVs_5_articulated_axle,HGVs_6_articulated_axle,all_HGVs,all_motor_vehicles
0,51,N,2004,2004-05-21,7,1,South West,E12000009,1,Isles of Scilly,...,1.0,13,0.0,0.0,0.0,0.0,0,0.0,0.0,19.0
1,51,N,2004,2004-05-21,8,1,South West,E12000009,1,Isles of Scilly,...,2.0,13,7.0,7.0,0.0,0.0,0,0.0,14.0,44.0
2,51,N,2004,2004-05-21,9,1,South West,E12000009,1,Isles of Scilly,...,5.0,17,5.0,0.0,0.0,0.0,0,0.0,5.0,45.0
3,51,N,2004,2004-05-21,10,1,South West,E12000009,1,Isles of Scilly,...,4.0,4,4.0,0.0,0.0,0.0,0,0.0,4.0,41.0
4,51,N,2004,2004-05-21,11,1,South West,E12000009,1,Isles of Scilly,...,3.0,14,6.0,0.0,0.0,0.0,0,0.0,6.0,41.0


In [5]:
pd_raw_count.columns

Index(['count_point_id', 'direction_of_travel', 'year', 'count_date', 'hour',
       'region_id', 'region_name', 'region_ons_code', 'local_authority_id',
       'local_authority_name', 'local_authority_code', 'road_name',
       'road_category', 'road_type', 'start_junction_road_name',
       'end_junction_road_name', 'easting', 'northing', 'latitude',
       'longitude', 'link_length_km', 'link_length_miles', 'pedal_cycles',
       'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
       'LGVs', 'HGVs_2_rigid_axle', 'HGVs_3_rigid_axle',
       'HGVs_4_or_more_rigid_axle', 'HGVs_3_or_4_articulated_axle',
       'HGVs_5_articulated_axle', 'HGVs_6_articulated_axle', 'all_HGVs',
       'all_motor_vehicles'],
      dtype='object')

In [6]:
pd_raw_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960860 entries, 0 to 4960859
Data columns (total 35 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   count_point_id                int64  
 1   direction_of_travel           object 
 2   year                          int64  
 3   count_date                    object 
 4   hour                          int64  
 5   region_id                     int64  
 6   region_name                   object 
 7   region_ons_code               object 
 8   local_authority_id            int64  
 9   local_authority_name          object 
 10  local_authority_code          object 
 11  road_name                     object 
 12  road_category                 object 
 13  road_type                     object 
 14  start_junction_road_name      object 
 15  end_junction_road_name        object 
 16  easting                       int64  
 17  northing                      int64  
 18  latitude              

In [7]:
# create a copy of the raw count data where region name is East of England
# pd_raw_count_east = pd_raw_count[pd_raw_count['region_name'] == 'East of England'].copy()

In [8]:
# pd_raw_count_east.shape # size of the dataframe

In [9]:
# pd_raw_count_east.info()

In [10]:
# pd_raw_count_east['direction_of_travel'].value_counts()

In [11]:
# pd_raw_count_east['year'].value_counts()

In [12]:
# pd_raw_count_east['local_authority_name'].value_counts() # local autorities wihtin the east of england region

In [13]:
pd_raw_count['road_type'].value_counts()

road_type
Minor    2714664
Major    2246196
Name: count, dtype: int64

In [14]:
pd_raw_count['year'].value_counts()

year
2008    347700
2009    308472
2019    301464
2018    289788
2003    222636
2004    216852
2006    212868
2002    211200
2007    206412
2010    204336
2005    201324
2001    193344
2012    188676
2013    188400
2011    187464
2017    186444
2000    181668
2021    166968
2016    166788
2015    161916
2014    157452
2022    157320
2020    151788
2023    149580
Name: count, dtype: int64

In [15]:
pd_raw_count['region_name'].value_counts()

region_name
South East                  792132
North West                  579684
East of England             550056
South West                  493632
West Midlands               480876
Yorkshire and the Humber    427968
London                      403560
East Midlands               398892
Scotland                    306096
Wales                       294432
North East                  233532
Name: count, dtype: int64

In [16]:
pd_raw_count['road_name'].value_counts()

road_name
U        1469808
C         673212
A1         37320
A38        34896
A6         23040
          ...   
B406          12
A3009         12
B4160         12
B4326         12
B3201         12
Name: count, Length: 3781, dtype: int64

# Data Formatting

In [17]:
# conver `count_date` to datetime type
pd_raw_count['count_date'] =  pd.to_datetime(pd_raw_count['count_date'])

In [18]:
# Extract Date Components 
pd_raw_count['Month'] = pd_raw_count['count_date'].dt.month
pd_raw_count['Day_of_Week'] = pd_raw_count['count_date'].dt.day_name()
pd_raw_count['Day'] = pd_raw_count['count_date'].dt.day

In [19]:
# Create a 'timestamp' column by combining 'count_date' and 'hour'
pd_raw_count['timestamp'] = pd.to_datetime(pd_raw_count['count_date']) + pd.to_timedelta(pd_raw_count['hour'], unit='h')

In [20]:
# Check data types
print("\nData types in pd_raw_count:")
print(pd_raw_count.dtypes)


Data types in pd_raw_count:
count_point_id                           int64
direction_of_travel                     object
year                                     int64
count_date                      datetime64[ns]
hour                                     int64
region_id                                int64
region_name                             object
region_ons_code                         object
local_authority_id                       int64
local_authority_name                    object
local_authority_code                    object
road_name                               object
road_category                           object
road_type                               object
start_junction_road_name                object
end_junction_road_name                  object
easting                                  int64
northing                                 int64
latitude                               float64
longitude                              float64
link_length_km                 

In [21]:
pd_raw_count.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,region_ons_code,local_authority_id,local_authority_name,...,HGVs_4_or_more_rigid_axle,HGVs_3_or_4_articulated_axle,HGVs_5_articulated_axle,HGVs_6_articulated_axle,all_HGVs,all_motor_vehicles,Month,Day_of_Week,Day,timestamp
0,51,N,2004,2004-05-21,7,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0,0.0,0.0,19.0,5,Friday,21,2004-05-21 07:00:00
1,51,N,2004,2004-05-21,8,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0,0.0,14.0,44.0,5,Friday,21,2004-05-21 08:00:00
2,51,N,2004,2004-05-21,9,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0,0.0,5.0,45.0,5,Friday,21,2004-05-21 09:00:00
3,51,N,2004,2004-05-21,10,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0,0.0,4.0,41.0,5,Friday,21,2004-05-21 10:00:00
4,51,N,2004,2004-05-21,11,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0,0.0,6.0,41.0,5,Friday,21,2004-05-21 11:00:00


# Data Cleaning

In [22]:
duplicates = pd_raw_count.duplicated().value_counts()
print(duplicates) 

False    4960860
Name: count, dtype: int64


In [23]:
print("\nMissing values in pd_raw_count:")
print((pd_raw_count.isna().sum() * 100 / pd_raw_count.shape[0]).round(2))


Missing values in pd_raw_count:
count_point_id                   0.00
direction_of_travel              0.00
year                             0.00
count_date                       0.00
hour                             0.00
region_id                        0.00
region_name                      0.00
region_ons_code                  0.00
local_authority_id               0.00
local_authority_name             0.00
local_authority_code             0.00
road_name                        0.00
road_category                    0.00
road_type                        0.00
start_junction_road_name        54.77
end_junction_road_name          54.77
easting                          0.00
northing                         0.00
latitude                         0.00
longitude                        0.00
link_length_km                  54.72
link_length_miles               54.72
pedal_cycles                     0.00
two_wheeled_motor_vehicles       0.00
cars_and_taxis                   0.00
buses_and_coaches

#### Filter rows with missing values

In [24]:
# Filter rows where either start_junction_road_name or end_junction_road_name has missing values
missing_roads = pd_raw_count[(pd_raw_count['start_junction_road_name'].isna()) | (pd_raw_count['end_junction_road_name'].isna())]

# Display the rows with missing values
print(missing_roads[['road_name', 'start_junction_road_name', 'end_junction_road_name']])

        road_name start_junction_road_name end_junction_road_name
110772       A174                      NaN                    NaN
110773       A174                      NaN                    NaN
110774       A174                      NaN                    NaN
110775       A174                      NaN                    NaN
110776       A174                      NaN                    NaN
...           ...                      ...                    ...
4960855     B4640                      NaN                    NaN
4960856     B4640                      NaN                    NaN
4960857     B4640                      NaN                    NaN
4960858     B4640                      NaN                    NaN
4960859     B4640                      NaN                    NaN

[2717232 rows x 3 columns]


#### List unique roads with missing junction names

In [27]:
# Get the unique road names where there are missing values
unique_roads_with_missing_values = pd_raw_count[(pd_raw_count['start_junction_road_name'].isna()) | (pd_raw_count['end_junction_road_name'].isna())]['road_name'].unique()

# Display the unique road names
print(unique_roads_with_missing_values)

['A174' 'A5038' 'A30' ... 'B317' 'B5380' 'B310']


In [116]:
# Drop the rows where 'link_length_km' and 'link_length_miles' have missing values
pd_raw_count_cleaned = pd_raw_count.dropna(subset=['link_length_km', 'link_length_miles'])

# Verify that there are no more missing values in these columns
missing_values_after_dropping = pd_raw_count_cleaned[['link_length_km', 'link_length_miles']].isna().sum()

# Display the remaining missing values (if any) and the shape of the cleaned dataset
(pd_raw_count_cleaned.shape, missing_values_after_dropping)

((2246196, 39),
 link_length_km       0
 link_length_miles    0
 dtype: int64)

In [29]:
pd_raw_count.shape

(4960860, 39)

In [30]:
columns_to_drop = [
    'year', 'region_id', 'local_authority_id', 'road_category', 'region_ons_code', 'road_name', 'local_authority_code',
    'easting', 'northing', 'start_junction_road_name', 'end_junction_road_name', 
    'link_length_km', 'link_length_miles', 'pedal_cycles', 
    'HGVs_2_rigid_axle', 'HGVs_3_rigid_axle', 'HGVs_4_or_more_rigid_axle', 
    'HGVs_3_or_4_articulated_axle', 'HGVs_5_articulated_axle', 'HGVs_6_articulated_axle', 
    'all_HGVs', 'buses_and_coaches', 'cars_and_taxis', 'two_wheeled_motor_vehicles', 
    'LGVs'
]

# east_england_data = pd_raw_count.drop(columns=columns_to_drop)

pd_raw_count.drop(columns=columns_to_drop, inplace=True)


In [31]:
pd_raw_count.sample(5)

,count_point_id,direction_of_travel,count_date,hour,region_name,local_authority_name,road_type,latitude,longitude,all_motor_vehicles,Month,Day_of_Week,Day,timestamp
2394322,803590,W,2020-05-18,17,South East,Kent,Minor,51.421859,0.378582,54.0,5,Monday,18,2020-05-18 17:00:00
3412710,941777,N,2000-05-08,13,North West,Manchester,Minor,53.475684,-2.176465,22.0,5,Monday,8,2000-05-08 13:00:00
1399526,48469,E,2009-09-29,9,North West,Cumbria,Major,54.331973,-2.739183,664.0,9,Tuesday,29,2009-09-29 09:00:00
691304,26777,W,2002-09-20,15,London,Lambeth,Major,51.482733,-0.114822,588.0,9,Friday,20,2002-09-20 15:00:00
3136410,931662,W,2006-06-19,13,East Midlands,Lincolnshire,Minor,52.659694,-0.493026,96.0,6,Monday,19,2006-06-19 13:00:00


In [35]:
pd_raw_count.shape

(4960860, 14)

In [33]:
pd_raw_count[['timestamp','count_point_id','direction_of_travel','road_type', 'latitude', 'longitude', 'all_motor_vehicles']].sort_values(by=['timestamp']).head()

,timestamp,count_point_id,direction_of_travel,road_type,latitude,longitude,all_motor_vehicles
918264,2000-03-17 07:00:00,36132,W,Major,51.427798,-2.532140,1850.0
658908,2000-03-17 07:00:00,26434,E,Major,51.535507,-0.343345,5459.0
1719552,2000-03-17 07:00:00,60046,S,Major,53.832745,-2.221801,334.0
1196748,2000-03-17 07:00:00,46072,S,Major,53.563702,-2.153642,2597.0
4506000,2000-03-17 07:00:00,952136,W,Minor,52.108388,-1.989427,43.0


In [36]:
print("\nMissing values in pd_raw_count:")
print((pd_raw_count.isna().sum() * 100 / pd_raw_count.shape[0]).round(2))


Missing values in pd_raw_count:
count_point_id          0.0
direction_of_travel     0.0
count_date              0.0
hour                    0.0
region_name             0.0
local_authority_name    0.0
road_type               0.0
latitude                0.0
longitude               0.0
all_motor_vehicles      0.0
Month                   0.0
Day_of_Week             0.0
Day                     0.0
timestamp               0.0
dtype: float64


In [37]:
duplicates = pd_raw_count.duplicated().value_counts()
print(duplicates) 

False    4960860
Name: count, dtype: int64


In [38]:
pd_raw_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960860 entries, 0 to 4960859
Data columns (total 14 columns):
 #   Column                Dtype         
---  ------                -----         
 0   count_point_id        int64         
 1   direction_of_travel   object        
 2   count_date            datetime64[ns]
 3   hour                  int64         
 4   region_name           object        
 5   local_authority_name  object        
 6   road_type             object        
 7   latitude              float64       
 8   longitude             float64       
 9   all_motor_vehicles    float64       
 10  Month                 int32         
 11  Day_of_Week           object        
 12  Day                   int32         
 13  timestamp             datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int32(2), int64(2), object(5)
memory usage: 492.0+ MB


In [39]:
# Save the DataFrame to a CSV file, ensuring the format is kept
pd_raw_count.to_csv('../data/pd_raw_count_revised_cleaned.csv', index=False)